In [2]:
from manim import *
import jupyter_capture_output
from numpy import linalg as npl

video_scene = " -v WARNING --disable_caching brown_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s brown_Scene"

Manim Community v0.18.0.post0

Jupyter Capture Output v0.0.11


In [3]:
# updates the state of one brownian particle
def next_state(state, params):
    L = params[0]
    x = state[0]
    y = state[1]
    x_increment = 2*np.random.randint(0, 2) - 1
    y_increment = 2*np.random.randint(0, 2) - 1
    return np.array([x + L*x_increment, y + L*y_increment, 0])


# calculate second moment of displacement
def get_second_displacement_moment(particle_list):
    sum = 0
    for particle in particle_list:    
        x = particle.state[0]
        y = particle.state[1]
        sum += x**2 + y**2
    return sum / len(particle_list)

In [4]:
%%capture_video --path "animations/brownian_motion/Brownian_Motion.mp4"
%%manim -qh --fps 60 $video_scene

class brown_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        headline = Title("Brownian Motion", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)  
        self.add(headline)

        
        # parameters
        origin = np.array([0, 0, 0])
        L_1D = 0.05
        L = np.sqrt(2) * L_1D
        params = (L_1D,)


        # add particles
        brown_1 = Circle(radius = 0.1, color = WHITE, fill_color = WHITE, fill_opacity = 0.5).move_to(origin)
        brown_2 = Circle(radius = 0.1, color = RED, fill_color = RED, fill_opacity = 0.5).move_to(origin)
        brown_3 = Circle(radius = 0.1, color = BLUE, fill_color = BLUE, fill_opacity = 0.5).move_to(origin)

        brown_1.state = np.array([*origin])
        brown_2.state = np.array([*origin])
        brown_3.state = np.array([*origin])

        brown_1.color = WHITE
        brown_2.color = RED
        brown_3.color = BLUE


        # displacement display
        second_moment = get_second_displacement_moment([brown_1, brown_2, brown_3])
        k_B_calculator_1 = MathTex("k_\mathrm{B}^\mathrm{exp}=k_\mathrm{B}^\mathrm{theo}\cdot").align_on_border(DOWN + LEFT)
        k_B_calculator_2 = Tex(f"{round(1/L**2*second_moment/1, 2)}").next_to(k_B_calculator_1, RIGHT)
        k_B_calculator_2.t = 0
        self.add(k_B_calculator_1, k_B_calculator_2)


        self.add(brown_1)
        self.add(brown_2)
        self.add(brown_3)
        self.wait(1.5)


        # updater to move a given brownian particle to its next position
        def particle_updater(particle):
            particle_state = particle.state                                 # current state of the particle
            particle_color = particle.color                                 # color of the particle 
            new_particle_state = next_state(particle_state, params)         # new state of the particle
            # connecting old and new position
            particle_connector = Line(start = particle_state, end = new_particle_state, color = particle_color, stroke_opacity = 0.5)
            self.add(particle_connector)
            # update the state of the particle to its new position
            particle.state = new_particle_state
            # move particle to new location
            particle.move_to(new_particle_state)


        def k_B_updater(k_B):
            k_B.t += 1
            if (k_B.t % 30 == 0):
                second_moment = get_second_displacement_moment([brown_1, brown_2, brown_3])
                k_B.become(Tex(f"{round(1/L**2*second_moment/k_B.t, 2)}").next_to(k_B_calculator_1, RIGHT))

        
        # add the updaters
        brown_1.add_updater(particle_updater)
        brown_2.add_updater(particle_updater)
        brown_3.add_updater(particle_updater)
        k_B_calculator_2.add_updater(k_B_updater)

        timeline = ValueTracker(0)
        self.play(timeline.animate.set_value(5), rate_func= linear, run_time = 30)
        self.wait(5)

Output saved by overwring previous file at animations/brownian_motion/Brownian_Motion.mp4.


<string>:34: SyntaxWarning: invalid escape sequence '\m'
